# Import libraries

In [5]:
import os
import pandas as pd
import re

# 1- Extract sheets from the excel files

In [22]:
# Define input and output folders
input_folder = "data"
velocity_output_folder = "extracted_data\\velocity"
acceleration_output_folder = "extracted_data\\acceleration"

# Make sure output folders exist
os.makedirs(input_folder, exist_ok=True)
os.makedirs(velocity_output_folder, exist_ok=True)
os.makedirs(acceleration_output_folder, exist_ok=True)

# Loop through each excel file
for file_name in os.listdir(input_folder):
    if file_name.endswith(".xlsx"):

        file_path = os.path.join(input_folder, file_name)
        xls = pd.ExcelFile(file_path)

        # Read the 'Segment Velocity' and 'Segment Acceleration' sheets
        velocity_df = xls.parse('Segment Velocity')
        acceleration_df = xls.parse('Segment Acceleration')

        # Create output paths
        velocity_output_path = os.path.join(
            velocity_output_folder, f"{file_name.split('.')[0]}-Velocity.csv")

        acceleration_output_path = os.path.join(
            acceleration_output_folder, f"{file_name.split('.')[0]}-Acceleration.csv")

        # Save the extracted data to a new csv file
        velocity_df.to_csv(velocity_output_path, index=False)
        acceleration_df.to_csv(acceleration_output_path, index=False)

        print(f"Processed and saved: {file_name}")

Processed and saved: MVN-J-Boning-64-001.xlsx
Processed and saved: MVN-J-Boning-64-002.xlsx
Processed and saved: MVN-J-Boning-64-003.xlsx
Processed and saved: MVN-J-Boning-64-004.xlsx
Processed and saved: MVN-J-Boning-64-005.xlsx
Processed and saved: MVN-J-Boning-64-006.xlsx
Processed and saved: MVN-J-Boning-79-001.xlsx
Processed and saved: MVN-J-Boning-90-001.xlsx
Processed and saved: MVN-J-Boning-90-002.xlsx
Processed and saved: MVN-J-Boning-90-003.xlsx
Processed and saved: MVN-J-Boning-90-004.xlsx
Processed and saved: MVN-J-Slicing-64-001.xlsx
Processed and saved: MVN-J-Slicing-73-001.xlsx
Processed and saved: MVN-J-Slicing-87-001.xlsx
Processed and saved: MVN-S-Boning-63-001.xlsx
Processed and saved: MVN-S-Boning-63-002.xlsx
Processed and saved: MVN-S-Boning-63-003.xlsx
Processed and saved: MVN-S-Boning-76-001.xlsx
Processed and saved: MVN-S-Boning-76-002.xlsx
Processed and saved: MVN-S-Boning-89-001.xlsx
Processed and saved: MVN-S-Boning-89-002.xlsx
Processed and saved: MVN-S-Boni

# 2 - Merge velocity and acceleration datasets

## 2.1 - Define folders

In [23]:
# Define input and output folders
velocity_folder = "extracted_data\\velocity"
acceleration_folder = "extracted_data\\acceleration"

# Make sure output folders exist
os.makedirs(velocity_folder, exist_ok=True)
os.makedirs(velocity_folder, exist_ok=True)

# Define folder for saving merged files
merged_output_folder = 'acc_vel_merged_data'
os.makedirs(merged_output_folder, exist_ok=True)  # Create the merged folder if it doesn't exist

## Handle edge cases

In [25]:
# Function to assign labels based on the Marker sheet
def assign_labels(frame_df, marker_df):
    labels = []
    
    # Loop through the marker dataframe
    for index, row in marker_df.iterrows():
        # Extract frame range and label number
        frame_range = row['Frame']  # e.g., '0-195'
        label = int(row['Label'].split()[0])  # Extract number from '3 - Reaching'
        
        # Split the frame range into start and end
        start_frame, end_frame = map(int, frame_range.split('-'))
        
        # Assign label to each frame in the range
        for frame in range(start_frame, end_frame + 1):
            labels.append((frame, label))
    
    # Convert the list of frame-label pairs into a DataFrame
    label_df = pd.DataFrame(labels, columns=['Frame', 'Label'])
    
    # Merge the labels with the original frame data
    merged_df = pd.merge(frame_df, label_df, on='Frame', how='left')
    
    return merged_df


In [33]:
def preprocess_files(velocity_file, acceleration_file, marker_file=None):
    # Load the velocity and acceleration files
    velocity_df = pd.read_csv(os.path.join(velocity_folder, velocity_file))
    acceleration_df = pd.read_csv(os.path.join(acceleration_folder, acceleration_file))

    # Handle boning-90-003: Copy Label from velocity to acceleration
    if 'boning-90-003' in velocity_file.lower():
        print(f"Fixing missing Label in acceleration sheet for: {acceleration_file}")
        acceleration_df['Label'] = velocity_df['Label']

    # Handle slicing-64-001: Rename Marker to Label
    if 'slicing-64-001' in velocity_file.lower():
        print(f"Renaming 'Marker' to 'Label' for: {velocity_file} and {acceleration_file}")
        velocity_df.rename(columns={'Marker': 'Label'}, inplace=True)
        acceleration_df.rename(columns={'Marker': 'Label'}, inplace=True)

    # Handle slicing-87-001: Add Label from Marker sheet
    if 'slicing-87-001' in velocity_file.lower():
        if marker_file is not None:
            print(f"Adding Label from Marker sheet for: {velocity_file} and {acceleration_file}")
            marker_df = pd.read_csv(marker_file)
            velocity_df = assign_labels(velocity_df, marker_df)
            acceleration_df = assign_labels(acceleration_df, marker_df)

    # Save the preprocessed data back to CSV files (overwrite or in new location)
    velocity_df.to_csv(os.path.join(velocity_folder, velocity_file), index=False)
    acceleration_df.to_csv(os.path.join(acceleration_folder, acceleration_file), index=False)

    return velocity_file, acceleration_file

In [35]:
velocity_files_to_preprocess = [
    'MVN-J-boning-90-003-Velocity.csv',
    'MVN-J-slicing-64-001-Velocity.csv',
    'MVN-J-slicing-87-001-Velocity.csv'
]

acceleration_files_to_preprocess = [
    'MVN-J-boning-90-003-Acceleration.csv',
    'MVN-J-slicing-64-001-Acceleration.csv',
    'MVN-J-slicing-87-001-Acceleration.csv'
]


# Preprocess the files
for velocity_file, acceleration_file in zip(velocity_files_to_preprocess, acceleration_files_to_preprocess):

    marker_file = None
    if 'slicing-87-001' in velocity_file.lower():
        marker_file = 'data\\MVN-J-Slicing-87-001-Marker.csv'  # Adjust this path to the actual Marker file
        
    preprocess_files(velocity_file, acceleration_file, marker_file)

Fixing missing Label in acceleration sheet for: MVN-J-boning-90-003-Acceleration.csv
Renaming 'Marker' to 'Label' for: MVN-J-slicing-64-001-Velocity.csv and MVN-J-slicing-64-001-Acceleration.csv
Adding Label from Marker sheet for: MVN-J-slicing-87-001-Velocity.csv and MVN-J-slicing-87-001-Acceleration.csv


## 2.2 - Function to merge acceleration and velocity

In [37]:
# Merge the velocity and acceleration data for each file
def merge_acc_vel(velocity_file, acceleration_file):
    # Read the velocity and acceleration data
    velocity_df = pd.read_csv(os.path.join(velocity_folder, velocity_file))
    acceleration_df = pd.read_csv(os.path.join(acceleration_folder, acceleration_file))

    # Check if the common columns are present in both dataframes
    common_cols = ['Frame', 'Label']
    if not all(column in velocity_df.columns and column in acceleration_df.columns for column in common_cols):
        print(f"Error: Common columns not found in {velocity_file} and {acceleration_file}. Skipping...")
        return

    # Merge the velocity and acceleration data on the common columns
    merged_df = pd.merge(velocity_df, acceleration_df, on=common_cols, suffixes=('_Vel', '_Acc'))

    # Save the merged dataframe as a new CSV file
    merged_filename = velocity_file.replace('-Velocity', '-Merged')
    merged_df.to_csv(os.path.join(merged_output_folder, merged_filename), index=False)

    print(f"Successfully merged and saved: {merged_filename}")

## 2.3 - Perform merging

In [38]:
# Get the list of all velocity and acceleration files
velocity_files = [f for f in os.listdir(velocity_folder) if f.endswith(".csv")]
acceleration_files = [f for f in os.listdir(acceleration_folder) if f.endswith(".csv")]

for velocity_file in velocity_files:
    # Find the corresponding acceleration file by replacing "Velocity" with "Acceleration"
    acceleration_file = velocity_file.replace("Velocity", "Acceleration")
    
    if acceleration_file in acceleration_files:
        merge_acc_vel(velocity_file, acceleration_file)
    else:
        print(f"Corresponding acceleration file not found for {velocity_file}. Skipping...")

Successfully merged and saved: MVN-J-Boning-64-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-002-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-003-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-004-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-005-Merged.csv
Successfully merged and saved: MVN-J-Boning-64-006-Merged.csv
Successfully merged and saved: MVN-J-Boning-79-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-001-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-002-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-003-Merged.csv
Successfully merged and saved: MVN-J-Boning-90-004-Merged.csv
Successfully merged and saved: MVN-J-Slicing-64-001-Merged.csv
Successfully merged and saved: MVN-J-Slicing-73-001-Merged.csv
Successfully merged and saved: MVN-J-Slicing-87-001-Merged.csv
Successfully merged and saved: MVN-S-Boning-63-001-Merged.csv
Successfully merged and saved: MVN-S-Boning-63-002-Merged.csv
Succe

### Verify merging

In [43]:
df = pd.read_csv('acc_vel_merged_data\\MVN-J-Slicing-87-001-Merged.csv')

df.columns

print ("YES" if 'Label' in df.columns else "NO")

print (df['Label'].unique())

YES
[3 5 4 8 2 7]


# 3 - Add knife category and shift column

In [55]:
"""
Extracts the knife sharpness value from the filename and categorizes it.
:param file_name: String, e.g., 'MVN-J-Boning-64-001_Acceleration.xlsx'
:return: Categorical label ('Blunt', 'Medium', 'Sharp')
"""
# Categorize knife sharpness
def add_sharpness_and_shift(file_name, df):
    # Extract knife sharpness factor from filename (e.g., '64' in 'MVN-J-Boning-64-001')
    knife_sharpness = int(file_name.split('-')[3])
    
    # Convert knife sharpness into categorical value
    if knife_sharpness >= 85:
        sharpness_category = 2  # Sharp
    elif 70 <= knife_sharpness <= 84:
        sharpness_category = 1  # Medium
    else:
        sharpness_category = 0  # Blunt
    
    # Add new column for sharpness category
    df['Knife_Sharpness_Category'] = sharpness_category

    # Extract the shift indicator from the filename (e.g., '001' in 'MVN-J-Boning-64-001')
    shift_indicator = file_name.split('-')[4]  # '001', '002', etc.
    
    # Add new column for the shift indicator
    df['Shift'] = shift_indicator
    
    return df

input_folder = "acc_vel_merged_data"
output_folder = "categorized_data"

os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

for file_name in os.listdir(input_folder):  
    if file_name.endswith(".csv"):

        file_path = os.path.join(input_folder, file_name)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Apply knife sharpness conversion function
        df = add_sharpness_and_shift(file_name, df)

        # Remove the "merged" annotation from the filename
        new_file_name = file_name.replace("-Merged", "")  # Replace "Merged" with an empty string

        # Save the updated DataFrame to a new CSV file with the knife sharpness column
        output_file_path = os.path.join(output_folder, new_file_name)
        df.to_csv(output_file_path, index=False)
        print(f"Processed and saved: {output_file_path}")

Processed and saved: categorized_data\MVN-J-Boning-64-001.csv
Processed and saved: categorized_data\MVN-J-Boning-64-002.csv
Processed and saved: categorized_data\MVN-J-Boning-64-003.csv
Processed and saved: categorized_data\MVN-J-Boning-64-004.csv
Processed and saved: categorized_data\MVN-J-Boning-64-005.csv
Processed and saved: categorized_data\MVN-J-Boning-64-006.csv
Processed and saved: categorized_data\MVN-J-Boning-79-001.csv
Processed and saved: categorized_data\MVN-J-Boning-90-001.csv
Processed and saved: categorized_data\MVN-J-Boning-90-002.csv
Processed and saved: categorized_data\MVN-J-Boning-90-003.csv
Processed and saved: categorized_data\MVN-J-Boning-90-004.csv
Processed and saved: categorized_data\MVN-J-Slicing-64-001.csv
Processed and saved: categorized_data\MVN-J-Slicing-73-001.csv
Processed and saved: categorized_data\MVN-J-Slicing-87-001.csv
Processed and saved: categorized_data\MVN-S-Boning-63-001.csv
Processed and saved: categorized_data\MVN-S-Boning-63-002.csv
Proce

In [54]:
# Test the outcome
df = pd.read_csv('categorized_data\\MVN-J-Slicing-87-001.csv')

print(df['Knife_Sharpness_Category'].unique())

print(df.shape)

[2]
(54721, 141)


# 4 - Merge different shift datasets for each knife sharpness and person

## Function to merge multiple CSV files for the same knife sharpness and activity

In [1]:
# Function to merge multiple CSV files for the same knife sharpness and activity
def merge_files_for_activity_and_sharpness(file_list, output_file):
    merged_df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as {output_file}")

## Group the files by activity and knife sharpness

In [8]:
# Group the files by activity and knife sharpness
def group_files_by_activity_and_sharpness(files):
    grouped_files = {}
    for file in files:
        # Extract knife sharpness factor (64, 79, etc.) and activity (Boning, Slicing) from the filename
        activity = file.split('-')[2]  # Extract activity (Boning, Slicing)
        sharpness = file.split('-')[3]    # Extract the knife sharpness factor
        
        # Use both activity and sharpness as keys for grouping
        key = f"{activity}"
        if key not in grouped_files:
            grouped_files[key] = []
        grouped_files[key].append(file)
    
    return grouped_files

## Main function to process and merge files for each person

In [9]:
# Main function to process and merge files for each person
def process_and_merge_person_files(input_folder, output_folder, person_indicator):
    os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists
    
    # Filter files for the specified person (J or S)
    person_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.startswith(f"MVN-{person_indicator}")]

    # Group the files by activity and knife sharpness
    grouped_files = group_files_by_activity_and_sharpness(person_files)

    # Merge files for each group (e.g., Boning-64, Slicing-87)
    for key, files in grouped_files.items():
        output_filename = os.path.join(output_folder, f"{person_indicator}_{key}.csv")
        merge_files_for_activity_and_sharpness(files, output_filename)

In [13]:
# Example usage to merge files for Person 1 (J) and Person 2 (S)
input_folder = "categorized_data"  # Folder containing all the raw CSV files
output_folder_person1 = "bone_slice"  # Folder to save merged files for Person 1
output_folder_person2 = "bone_slice"  # Folder to save merged files for Person 2

# Merge files for Person 1 (J)
process_and_merge_person_files(input_folder, output_folder_person1, "J")

# Merge files for Person 2 (S)
process_and_merge_person_files(input_folder, output_folder_person2, "S")

Merged file saved as workers\J_Boning.csv
Merged file saved as workers\J_Slicing.csv
Merged file saved as workers\S_Boning.csv
Merged file saved as workers\S_Slicing.csv


# 5 - Adjusting the class labels to make them consistent for merging

In [14]:
# Load the boning dataset
boning_df_P1 = pd.read_csv('bone_slice\\J_Boning.csv')
boning_df_P2 = pd.read_csv('bone_slice\\S_Boning.csv')

# Adjust the 'Dropping' class label from 5 to 8
boning_df_P1['Label'] = boning_df_P1['Label'].replace({5: 8})
boning_df_P2['Label'] = boning_df_P2['Label'].replace({5: 8})

# Save the updated boning dataset
boning_df_P1.to_csv('bone_slice\\J_Boning.csv', index=False)
boning_df_P2.to_csv('bone_slice\\S_Boning.csv', index=False)

print("Class labels in the boning dataset have been updated.")

Class labels in the boning dataset have been updated.


In [17]:
# Check the class labels in the boning dataset
print(boning_df_P1['Label'].unique())
print(boning_df_P2['Label'].unique())

[4 0 2 3 8 1]
[4 8 1 2 0 3]


In [18]:
boning_df_P1.shape

(141397, 142)

# 6 - Convert to 1-minute interval 

In [68]:
# Function to convert 1-second data to 1-minute intervals
def convert_to_one_minute_intervals(df):
    # Create a 'Minute' column by dividing frames into groups of 60
    if 'Minute' in df.columns:
        df.drop(columns=['Minute'], inplace=True)

    # Create a global frame counter based on the total number of rows
    df['Global_Frame'] = df.index + 1  # Using the index as the frame number (starting from 1)
    
    # Create the 'Minute' column by dividing Global_Frame into groups of 60 seconds
    df['Minute'] = df['Global_Frame'] // 60

    # Define the columns for categorical data
    categorical_columns = ['Label', 'Knife_Sharpness_Category', 'Shift']

    # Define the columns for numerical data (excluding 'Frame')
    numerical_columns = [col for col in df.columns if col not in categorical_columns and col not in ['Frame', 'Global_Frame', 'Minute']]

    # Aggregation rules
    aggregation = {}

    # Set aggregation rules for categorical columns to take the mode (most frequent value)
    for col in categorical_columns:
        aggregation[col] = lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0]  # Handle empty mode case

    # Set aggregation rules for numerical columns to take the mean
    for col in numerical_columns:
        aggregation[col] = 'mean'
    
    # Group by the 'Minute' column and apply the aggregation rules
    aggregated_df = df.groupby('Minute').agg(aggregation)

    # Reset the index of the aggregated DataFrame
    # aggregated_df.reset_index(inplace=True)

    if 'Frame' in df.columns:
        df.drop(columns=['Frame'], inplace=True)

     # Drop the 'Global_Frame' column after conversion
    if 'Global_Frame' in aggregated_df.columns:
        aggregated_df.drop(columns=['Global_Frame'], inplace=True)

    return aggregated_df

In [69]:
# Load your dataset (assuming it's already been merged and processed)
boning_P1 = pd.read_csv('bone_slice\\J_Boning.csv')
boning_P2 = pd.read_csv('bone_slice\\S_Boning.csv')
slicing_P1 = pd.read_csv('bone_slice\\J_Slicing.csv')
slicing_P2 = pd.read_csv('bone_slice\\S_Slicing.csv')

print(boning_P1.shape)
print(boning_P2.shape)
print(slicing_P1.shape)
print(slicing_P2.shape)

# Convert the dataset to 1-minute intervals
boning_p1_min = convert_to_one_minute_intervals(boning_P1)
boning_p2_min = convert_to_one_minute_intervals(boning_P2)
slicing_p1_min = convert_to_one_minute_intervals(slicing_P1)
slicing_p2_min = convert_to_one_minute_intervals(slicing_P2)

print(boning_p1_min.shape)
print(boning_p2_min.shape)
print(slicing_p1_min.shape)
print(slicing_p2_min.shape)

# Save the 1 minute interval datasets
boning_p1_min.to_csv('bone_slice_min\\J_Boning_min.csv')
boning_p2_min.to_csv('bone_slice_min\\S_Boning_min.csv')
slicing_p1_min.to_csv('bone_slice_min\\J_Slicing_min.csv')
slicing_p2_min.to_csv('bone_slice_min\\S_Slicing_min.csv')

print("Conversion to 1-minute intervals completed.")

(141397, 142)
(137732, 142)
(125527, 142)
(125637, 142)
(2357, 141)
(2296, 141)
(2093, 141)
(2094, 141)
Conversion to 1-minute intervals completed.


In [74]:
print (boning_p1_min.head)

<bound method NDFrame.head of         Label  Knife_Sharpness_Category  Shift  Pelvis x_Vel  Pelvis y_Vel  \
Minute                                                                       
0           4                         0      1           0.0           0.0   
1           4                         0      1           0.0           0.0   
2           4                         0      1           0.0           0.0   
3           0                         0      1           0.0           0.0   
4           0                         0      1           0.0           0.0   
...       ...                       ...    ...           ...           ...   
2352        4                         2      4           0.0           0.0   
2353        4                         2      4           0.0           0.0   
2354        4                         2      4           0.0           0.0   
2355        8                         2      4           0.0           0.0   
2356        8                     

In [89]:
# Check for columns that are present in the minutes dataset but not in the original dataset
def check_missing_columns(original_df, minutes_df):
    missing_columns = [col for col in minutes_df.columns if col not in original_df.columns]
    return missing_columns


print("Missing columns in the boning dataset:")
print(check_missing_columns(boning_P1, boning_p1_min))

print(slicing_p2_min.index)

Missing columns in the boning dataset:
[]
Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2084, 2085, 2086, 2087, 2088, 2089, 2090, 2091, 2092, 2093],
      dtype='int64', name='Minute', length=2094)


# 7 - Merge Boning and Slicing

In [ ]:
import pandas as pd

# Load the converted datasets (boning and slicing) for the same person
boning_df_p1 = pd.read_csv('bone_slice_min/J_Boning_min.csv')
boning_df_p2 = pd.read_csv('bone_slice_min/S_Boning_min.csv')
slicing_df_p1 = pd.read_csv('bone_slice_min/J_Slicing_min.csv')
slicing_df_p2 = pd.read_csv('bone_slice_min/S_Slicing_min.csv')

# Introduce the 'Main_Activity' column
boning_df_P1['Main_Activity'] = '0' # 0 for Boning
boning_df_P2['Main_Activity'] = '0'
slicing_df_p1['Main_Activity'] = '1' # 1 for Slicing
slicing_df_p2['Main_Activity'] = '1' 

# Merge the boning and slicing datasets (row-wise)
merged_df_p1 = pd.concat([boning_df_p1, slicing_df_p1], ignore_index=True)
merged_df_p2 = pd.concat([boning_df_p2, slicing_df_p2], ignore_index=True)

# Ensure 'Main_Activity' column has no NaN values
merged_df_p1['Main_Activity'] = merged_df_p1['Main_Activity'].fillna('0')  
merged_df_p2['Main_Activity'] = merged_df_p2['Main_Activity'].fillna('0')  

# After merging, convert 'Label' back to integer
merged_df_p1['Label'] = merged_df_p1['Label'].astype('Int64')  # Use 'Int64' to handle NaN if any
merged_df_p2['Label'] = merged_df_p2['Label'].astype('Int64')


# Save the merged dataset
merged_df_p1.to_csv('bone_slice_merged/Worker1.csv', index=False)
merged_df_p2.to_csv('bone_slice_merged/Worker2.csv', index=False)

print("Boning and Slicing datasets have been successfully merged.")


Boning and Slicing datasets have been successfully merged.


In [93]:
print(merged_df_p1.info)
print(merged_df_p2.shape)

print(merged_df_p1.columns)

<bound method DataFrame.info of       Minute  Label  Knife_Sharpness_Category  Shift  Pelvis x_Vel  \
0          0      4                         0      1           0.0   
1          1      4                         0      1           0.0   
2          2      4                         0      1           0.0   
3          3      0                         0      1           0.0   
4          4      0                         0      1           0.0   
...      ...    ...                       ...    ...           ...   
4445    2088      8                         2      1           0.0   
4446    2089      3                         2      1           0.0   
4447    2090      3                         2      1           0.0   
4448    2091      3                         2      1           0.0   
4449    2092      3                         2      1           0.0   

      Pelvis y_Vel  Pelvis z_Vel  L5 x_Vel  L5 y_Vel  L5 z_Vel  ...  \
0              0.0           0.0  0.027806 -0.051742  0.

# 8 - Merge P1 and P2

In [95]:
# Merge person1.csv and person2.csv
person1_df = pd.read_csv('bone_slice_merged/Worker1.csv')
person2_df = pd.read_csv('bone_slice_merged/Worker2.csv')

# introduce new column 'Worker' to identify the person
person1_df['Worker'] = 1
person2_df['Worker'] = 2

# Merge the datasets for Worker 1 and Worker 2
merged_df = pd.concat([person1_df, person2_df], ignore_index=True)

# Save the merged dataset
merged_df.to_csv('bone_slice_merged/AllWorkers.csv', index=False)

print ("Merged dataset saved as 'AllWorkers.csv'.")

print(merged_df.shape)
print(merged_df.info)

Merged dataset saved as 'AllWorkers.csv'.
(8840, 144)
<bound method DataFrame.info of       Minute  Label  Knife_Sharpness_Category  Shift  Pelvis x_Vel  \
0          0      4                         0      1           0.0   
1          1      4                         0      1           0.0   
2          2      4                         0      1           0.0   
3          3      0                         0      1           0.0   
4          4      0                         0      1           0.0   
...      ...    ...                       ...    ...           ...   
8835    2089      4                         2      1           0.0   
8836    2090      4                         2      1           0.0   
8837    2091      4                         2      1           0.0   
8838    2092      7                         2      1           0.0   
8839    2093      7                         2      1           0.0   

      Pelvis y_Vel  Pelvis z_Vel  L5 x_Vel  L5 y_Vel  L5 z_Vel  ...  \
0 